In [1]:
import torch
import sys
import os
from torchvision.datasets.folder import pil_loader
from torchvision.transforms import Normalize, Compose, RandomHorizontalFlip, RandomRotation, Resize, ToTensor, ToPILImage
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
device = torch.device('cuda')

In [3]:
device

device(type='cuda')

In [4]:
path = os.path.abspath(os.path.join('..'))

In [5]:
if path not in sys.path:
    sys.path.append(path+"/mish_cuda/")

In [6]:
from mish_cuda.mish import MishCuda

In [7]:
if path not in sys.path:
    sys.path.append(path)

In [8]:
sys.path

['/home/adtp/github/adtp/ScaledYOLOv4/notebooks',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python38.zip',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/lib-dynload',
 '',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages/IPython/extensions',
 '/home/adtp/.ipython',
 '/home/adtp/github/adtp/ScaledYOLOv4/mish_cuda/',
 '/home/adtp/github/adtp/ScaledYOLOv4']

In [9]:
import models

### Turn model checkpoint into state_dict
useful to use our Mish instead of MishCuda

In [10]:
save_scaled = torch.load('best_2piece_equilibree.pt', map_location=device)

In [11]:
weight_scaled = save_scaled['model'].state_dict()

In [12]:
torch.save(weight_scaled, 'best_2piece_equilibree.pth')

In [ ]:
weights_scaled_load = torch.load('best.pth')

In [ ]:
save_scaled['model'].load_state_dict(weights_scaled_load)

In [ ]:
python detect.py --weights ../weights/yolov4-p7.pth --cfg data/coco.yaml --source ../person.jpg

## Load model

In [ ]:
from models.yolo import Model

In [ ]:
net = Model(cfg='../models/yolov4-csp.yaml', ch=3, nc=80)

In [ ]:
weights_scaled = torch.load('/home/adtp/github/adtp/weights/Trainings/Must/best_New_data_1_2.pth')

In [ ]:
net.load_state_dict(weights_scaled)

## Experiment inference

In [ ]:
net.to(device)
net.eval();

In [ ]:
# img_test = ToTensor()(Resize((416, 416))(pil_loader('bradley-cooper.png'))).unsqueeze_(0)
# img_test = ToTensor()(Resize((800, 1280))(pil_loader('street.jpg'))).unsqueeze_(0)
img_test = ToTensor()(Resize((640, 640))(pil_loader('/home/adtp/github/adtp/street.jpg'))).unsqueeze_(0)

In [ ]:
pred = net(img_test.to(device))

In [ ]:
pred[0].shape

In [ ]:
torch.argmax(pred[0][:, :, 5:], dim=2)

In [ ]:
with torch.no_grad():
    outputs = net(images)
predicted = torch.argmax(outputs, 1)